# Collecter des données avec l'API de Bluesky

## Découvrir l'API

Jeter un coup d'oeil ici : https://docs.bsky.app/docs/category/http-reference

Installer la bibliothèque atproto

In [2]:
# pip install atproto

Charger les bibliothèques

In [1]:
from atproto import Client
import yaml
import pandas as pd

Se connecter

In [2]:
with open("creds.yaml", "r") as f:
    creds = yaml.load(f, Loader=yaml.FullLoader)
client = Client()
client.login(creds["username"], creds["password"])

ProfileViewDetailed(did='did:plc:svofadxzzkfjjcjh3xzhy73v', handle='activetigger.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated', activitySubscription={'allowSubscriptions': 'followers'}), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:svofadxzzkfjjcjh3xzhy73v/bafkreieocqwf3shlt5uzpm6rf26z36utigo5ktwlqqppkvhkotu3yqmkxe@jpeg', banner=None, created_at='2025-03-09T16:02:10.241Z', description=None, display_name='', followers_count=1, follows_count=1, indexed_at='2025-03-09T16:02:10.241Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=4, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewDetailed')

Envoyer votre premier post

In [6]:
post = client.send_post('Hello world! I posted this via the Python SDK.')

## Récupérer des postes

### Structure des données d'un post

https://bsky.app/profile/dorialexander.bsky.social/post/3lkiuwyhtcs2z

Pour cela il faut récupérer le did du post et son identifiant

In [7]:
url = "https://bsky.app/profile/bdecourson.bsky.social/post/3meiwcx6jts26"
handle = "bdecourson.bsky.social"
post_id = "3meiwcx6jts26"

Faire une regex ?

Résoudre le handle de l'utilisateur

In [8]:
user_info = client.resolve_handle(handle)
did = user_info.did
did, user_info

('did:plc:b67oujcjw4yhqzbkgkmfxkxw',
 Response(did='did:plc:b67oujcjw4yhqzbkgkmfxkxw'))

Obtenir le post

In [9]:
post_uri = f"at://{did}/app.bsky.feed.post/{post_id}"
post = client.app.bsky.feed.get_post_thread({"uri":post_uri})
post

Response(thread=ThreadViewPost(post=PostView(author=ProfileViewBasic(did='did:plc:b67oujcjw4yhqzbkgkmfxkxw', handle='bdecourson.bsky.social', associated=ProfileAssociated(chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=None, labeler=None, lists=None, starter_packs=None, py_type='app.bsky.actor.defs#profileAssociated', activitySubscription={'allowSubscriptions': 'followers'}), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:b67oujcjw4yhqzbkgkmfxkxw/bafkreifaasm23s25mbq2i5xq3qzozziv7pniqa3xg5cekzwvyrzt5biyuu@jpeg', created_at='2023-09-29T23:22:13.880Z', display_name='Benoît de Courson', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreifx4fnvtcqyzngfhdzzhn3bzjwc6jtra5yiqaxdmx2wf6rjpb26ve

In [10]:
post.model_dump()

{'thread': {'post': {'author': {'did': 'did:plc:b67oujcjw4yhqzbkgkmfxkxw',
    'handle': 'bdecourson.bsky.social',
    'associated': {'chat': {'allow_incoming': 'following',
      'py_type': 'app.bsky.actor.defs#profileAssociatedChat'},
     'feedgens': None,
     'labeler': None,
     'lists': None,
     'starter_packs': None,
     'py_type': 'app.bsky.actor.defs#profileAssociated',
     'activitySubscription': {'allowSubscriptions': 'followers'}},
    'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:b67oujcjw4yhqzbkgkmfxkxw/bafkreifaasm23s25mbq2i5xq3qzozziv7pniqa3xg5cekzwvyrzt5biyuu@jpeg',
    'created_at': '2023-09-29T23:22:13.880Z',
    'display_name': 'Benoît de Courson',
    'labels': [],
    'viewer': {'blocked_by': False,
     'blocking': None,
     'blocking_by_list': None,
     'followed_by': None,
     'following': None,
     'known_followers': None,
     'muted': False,
     'muted_by_list': None,
     'py_type': 'app.bsky.actor.defs#viewerState'},
    'py_type': 'a

Explorer le contenu d'un post et ce qu'on a

Astuce : l'écrire dans un fichier

In [14]:
with open("post.json", "w") as f:
    f.write(post.json())

Savoir qui l'a reposté/apprécié ?

app.bsky.feed.getRepostedBy


In [13]:
reposted = client.app.bsky.feed.get_reposted_by({"uri":post_uri})
len(reposted.model_dump()["reposted_by"])

### Récupérer tous les postes d'un compte

In [11]:
all_posts = []
cursor = None

while True:
    response = client.app.bsky.feed.get_author_feed({"actor":did, "limit":100, "cursor":cursor})
    all_posts.extend(response["feed"])
    cursor = response.cursor
    if not cursor:
        break  # Fin de la pagination

print(f"Nombre total de posts récupérés : {len(all_posts)}")

# Optionnel : Afficher les 5 premiers posts
for post in all_posts[:5]:
    print(post["post"]["record"]["text"])
    

Nombre total de posts récupérés : 867
Oui !
Si ça vous intéresse de mesurer ça, ça me prendrait un gros quart d'heure. Et on peut raffiner, par exemple classer les personnes par genre avec leur prénom.
Je ne trouve plus les transcriptions sur votre site mais il me semble qu'elles existent ?
Oh. Oui la reconnaissance d'entités nommées c'est une des taches qui s'automatise le mieux. Petite démo (il faut choisir un modèle en français) 
demos.explosion.ai/displacy-ent...
Une auto-lexicométrie, c'est beau !
Avec spacy, vous pourriez regarder en deux temps trois mouvements quelles personnes (historien.ne.s ou personnages historiques) sont les plus mentionnées
Merci !!


### Récupérer tous les postes d'un feed

Ex. feed python https://bsky.app/profile/clairem.secondlife.bio/feed/aaajf4zkrdcte

In [16]:
user_info = client.resolve_handle("clairem.secondlife.bio")
did = user_info.did
feed_id = "aaajf4zkrdcte"
feed_uri = f"at://{did}/app.bsky.feed.generator/{feed_id}"
print(feed_uri)

# Retrieve all posts from the feed
all_posts = []
cursor = None

while True:
    # Request feed posts with pagination
    response = client.app.bsky.feed.get_feed({"feed":feed_uri, "limit":100, "cursor":cursor})

    # Add posts to the list
    all_posts.extend(response["feed"])

    # Handle pagination
    cursor = response.cursor
    if not cursor:
        break  # No more posts to fetch

# Print total posts retrieved
print(f"Total posts retrieved from feed '{feed_uri}': {len(all_posts)}")

at://did:plc:te235k5t6h4hp3awy5tuajxx/app.bsky.feed.generator/aaajf4zkrdcte
Total posts retrieved from feed 'at://did:plc:te235k5t6h4hp3awy5tuajxx/app.bsky.feed.generator/aaajf4zkrdcte': 977


In [34]:
print(all_posts[10].dict()["post"]["record"]["text"])

[patrick] $: python feckoff.py


### Faire une recherche

https://atproto.blue/en/latest/atproto/atproto_client.models.app.bsky.feed.search_posts.html

In [3]:
all_posts = []
cursor = None
n_elements = 10000
while True: 
    response = client.app.bsky.feed.search_posts({"q":"computational social science", "limit":100, "sort": 'latest',"cursor":cursor})
    all_posts.extend(response["posts"])
    cursor = response.cursor
    if not cursor:
        break  # No more posts to fetch
    if len(all_posts) >= n_elements:
        break
    print(len(all_posts))


99
196
296
394
493
593
684
784
884
979
1079
1176
1274
1374
1474
1573


## Mettre en forme les données

- Construire un dataframe avec les champs d'intérêt
- Le sauvegarder

In [12]:
post = all_posts[1100]

In [13]:
post.model_dump()

{'author': {'did': 'did:plc:zctue4vtjwztlygf2lu7ydtm',
  'handle': 'daviddarmofal.bsky.social',
  'associated': {'chat': None,
   'feedgens': None,
   'labeler': None,
   'lists': None,
   'starter_packs': None,
   'py_type': 'app.bsky.actor.defs#profileAssociated',
   'activitySubscription': {'allowSubscriptions': 'followers'}},
  'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:zctue4vtjwztlygf2lu7ydtm/bafkreif64p5se53o4y2mjstdi7g3zy5a4cd247wcbthfqh5qsv2fdksryq@jpeg',
  'created_at': '2023-06-14T13:36:23.830Z',
  'display_name': 'David Darmofal',
  'labels': [{'cts': '1970-01-01T00:00:00.000Z',
    'src': 'did:plc:zctue4vtjwztlygf2lu7ydtm',
    'uri': 'at://did:plc:zctue4vtjwztlygf2lu7ydtm/app.bsky.actor.profile/self',
    'val': '!no-unauthenticated',
    'cid': 'bafyreichz227fxjsrv7si2z7rncmf7h5q5ucucn5aypbg3r6roohzn7goy',
    'exp': None,
    'neg': None,
    'sig': None,
    'ver': None,
    'py_type': 'com.atproto.label.defs#label'}],
  'viewer': {'blocked_by': False,
  

In [16]:
df = pd.DataFrame([[post.record.created_at, post.record.text, post.author.did, 
                    post.author.handle, post.author.display_name, post.author.created_at, 
                    post.reply_count, post.like_count, post.repost_count]
 for post in all_posts], columns=["created_at", "text", "author_did", "author_handle", 
                                  "author_display_name", "author_created_at", "reply_count", "like_count", "repost_count"])
df["created_at"] = pd.to_datetime(df["created_at"], format="mixed", utc=True)
df.set_index("created_at", inplace=True)
df.to_csv("./data/bsky_posts.csv")


## Débuter une analyse

- Post les plus influents
- Auteurs les plus influents
- Analyser le profil des auteurs les plus influents (automatiser la récupération de leurs données)
- Extension des mots clés